# Notebook to evaluate inferred microbial anomaly scores

To run this notebook you need to create and activate the following conda environment:

```
conda create --name score_eval -c conda-forge -c defaults numpy pandas matplotlib seaborn scipy scikit-learn ipython ipykernel -y
conda activate score_eval
pip install -e .
```


## Setup

In [ ]:
import os
import warnings

import matplotlib.pyplot as plt
import pandas as pd

from src.utils_eval_score import (
    _get_ordinal_suffix,
    _plot_score_after_nth_abx_exposure,
    _plot_score_over_age,
    _select_samples_around_nth_abx_exposure,
    display_scatterplot_w_scores,
    get_scores_n_abx_info,
    plot_time_between_abx_exposures,
    plot_trajectory,
)
from src.utils_meta import display_diet_information

%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams.update({"font.family": "DejaVu Sans"})
plt.style.use("tableau-colorblind10")

warnings.filterwarnings("ignore", category=FutureWarning)
# avg. number of days per month
DAYS_PER_MONTH = 30.437

USER input: define the inferred model and linked datasets to evaluate here:

In [ ]:
#### USER INPUT START
# name of the model
model_name = "saved_models_microbial_novel_alpha_div2/id-55"
# which model version to evaluate: "best" or "last"
point_to_evaluate = "best"

# name of feature dataset used for model
ft_name = "ft_vat19_anomaly_v20240806_entero_genus"
# name of abx time-series used for model
abx_ts_name = "ts_vat19_abx_v20240806"

# limit evaluation to time range up to this many months (if None no limit is set
# and all scores are evaluated)
limit_months = 24.0

# whether to group samples prior to abx exposure in analysis
group_samples = True

# how many samples prior and after abx exposure to consider
min_samples = -3.0
max_samples = 6.0

# whether to filter noabx score samples by having at least 1 obs prior to cutoff
no_filter = True

# whether to display diet info after 1st, 2nd, 3rd abx exposure plots
display_diet_info = True

# whether to have max. resolution of 0.5 months or not
max_resolution = False

# scaling factor options:
scaling_factors_used = True

# if scaling_factors_used is True, then the following options are required:
# non-centered = "nc_std" or centered = "std"
stddev_type = "nc_std"
# moving average window size: 30 or 10
moving_avg = 10
# whether to include duplicates: "--RD-True" or ""
duplicates = "--RD-False"
# using lower bound of 1 for SFs: "lower_bound-1" or ""
lower_bound = ""

#### USER INPUT END

## Prepare data

In [ ]:
base_path = f"../data/{model_name}/anomaly_detection/"

res_n_group = f"g{str(group_samples)[0]}_maxres{str(max_resolution)[0]}"

if scaling_factors_used:
    print("Scaling factors used.")
    folder_name = f"using-SF_{stddev_type}_z_scores--moving_avg-{moving_avg}-cummax{lower_bound}{duplicates}"

    scores_path = f"{base_path}scores_{point_to_evaluate}_normal/{folder_name}/"
    evaluation_path = f"{base_path}evaluation_{point_to_evaluate}_overall_{res_n_group}_{stddev_type}_ma{moving_avg}{duplicates.replace('-', '_').lower()}/"
else:
    scores_path = f"{base_path}scores_{point_to_evaluate}_normal/"
    evaluation_path = (
        f"{base_path}evaluation_{point_to_evaluate}_overall_{res_n_group}_no_scaling/"
    )
if not os.path.exists(evaluation_path):
    os.makedirs(evaluation_path)


In [ ]:
noabx_train, noabx_val, abx_scores_flat, abx_df, abx_age_at_all = get_scores_n_abx_info(
    scores_path, ft_name, limit_months, abx_ts_name, no_filter=no_filter
)


## Score after abx exposure 1st, 2nd and 3rd


### score_1, score_2 and score_3 respectively

In [ ]:
# get samples around n-th abx exposure
for n in [1, 2, 3]:
    score_col = f"score_{n}"
    scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
        abx_scores_flat,
        abx_df,
        n=n,
        min_samples=min_samples,
        max_samples=max_samples,
        group_samples=group_samples,
        score_var=score_col,
        max_resolution=max_resolution,
    )
    print(scores_abx_nth_samples.shape)
    _plot_score_after_nth_abx_exposure(
        scores_abx_nth_samples,
        x_axis="diff_age_nth_abx",
        y_axis=score_col,
        n=n,
        path_to_save=evaluation_path,
        flag=score_col,
        min_samples=min_samples,
        max_samples=max_samples,
        max_resolution=max_resolution,
        grouped_samples=group_samples,
    )

    if display_diet_info:
        suff = _get_ordinal_suffix(n)
        xlabel = f"Months since {n}{suff} abx exposure"
        if group_samples:
            if max_resolution:
                t1_reference = -0.5
            else:
                t1_reference = -1.0
            xlabel += f"\n\n(Here {t1_reference} is last sample prior "
            xlabel += f"to abx since {min_samples} months)"
        display_diet_information(
            scores_abx_nth_samples,
            "diet_milk",
            "diff_age_nth_abx",
            "samples",
            f"Diet covariates in samples before/after {n}{suff} abx exposure",
            x_axis=xlabel,
            path_to_save=evaluation_path,
        )
        display_diet_information(
            scores_abx_nth_samples,
            "diet_weaning",
            "diff_age_nth_abx",
            "samples",
            f"Diet covariates in samples before/after {n}{suff} abx exposure",
            x_axis=xlabel,
            path_to_save=evaluation_path,
        )

In [ ]:
plot_time_between_abx_exposures(
    abx_age_at_all, n0_label="1st", n1_label="2nd", path_to_save=evaluation_path
)
plt.show()
plot_time_between_abx_exposures(
    abx_age_at_all, n0_label="2nd", n1_label="3rd", path_to_save=evaluation_path
)
plt.show()

Funfact: Here we compare hosts that have at least one sample prior or after the n-th abx exposure - so a sample that is directly influenced by the n-th exposure. This means concretely for n = 1:
* there are 141 hosts with a 1st abx exposure
* however, only 109 hosts are displayed in our plot with the filtering min_samples=-3, max_samples=6 and limit_months=24. 20 hosts are disregarded due to these filters. the remaining 12 hosts (see list below) have no samples prior or after the 1st abx exposure and are always excluded from these plots.

list of hosts with no sample before or directly affected by 1st abx exposure only: `['P003825', 'E002825', 'E007743', 'E014953', 'T029922', 'P010096', 'T000094', 'T000272', 'E012971', 'T028183', 'E016444', 'T007404']`

## Compare characteristics of 1st vs. 2nd abx exposures

In [ ]:
results = []

for n in [1, 2, 3]:
    score_col = f"score_{n}"
    scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
        abx_scores_flat,
        abx_df,
        n=n,
        min_samples=min_samples,
        max_samples=max_samples,
        group_samples=group_samples,
        score_var=score_col,
        max_resolution=max_resolution,
    )
    # number of unique infants exposed
    n_infants = scores_abx_nth_samples["host_id"].nunique()
    # Mean and std of abx duration
    abx_duration_desc = scores_abx_nth_samples["abx_any_last_dur_days"].describe()
    duration_mean = abx_duration_desc["mean"]
    duration_std = abx_duration_desc["std"]

    # Top x abx_type names
    abx_type_top = (
        scores_abx_nth_samples["abx_type"]
        .value_counts(dropna=False)
        .head(2)
        .index.tolist()
    )

    # Top x abx_reason names
    abx_reason_top = (
        scores_abx_nth_samples["abx_reason"]
        .value_counts(dropna=False)
        .head(2)
        .index.tolist()
    )

    # infant age at abx administration: age_nth_abx
    age_desc = scores_abx_nth_samples["age_nth_abx"].describe()

    # Collect results
    results.append(
        {
            "Cumcount abx exposure": n,
            # "nb_infants": n_infants,
            "age_mean": age_desc["mean"],
            "age_std": age_desc["std"],
            "abx_duration_days_mean": abx_duration_desc["mean"],
            "abx_duration_days_std": abx_duration_desc["std"],
            "abx_types_top2": abx_type_top,
            "abx_reasons_top2": abx_reason_top,
        }
    )

abx_stats_df = pd.DataFrame(results)
numeric_cols = [
    x for x in abx_stats_df.columns if x.endswith("_mean") or x.endswith("_std")
]
abx_stats_df[numeric_cols] = abx_stats_df[numeric_cols].round(2)
abx_stats_df

Their abx characteristics are similar.

## Score after 3rd abx: split by time since 2nd abx exposure


In [ ]:
if not group_samples:
    duration_threshold = 4  # in months

    # perform grouping
    abx_time_between = abx_age_at_all.copy()
    abx_time_between["time_since_2nd"] = (
        abx_time_between["age_3rd_abx"] - abx_time_between["age_2nd_abx"]
    )

    bins = [-float("inf"), duration_threshold, float("inf")]
    between_labels = [
        f"<= {duration_threshold} months",
        f"> {duration_threshold} months",
    ]
    abx_time_between["time_since_2nd_cat"] = pd.cut(
        abx_time_between["time_since_2nd"],
        bins=bins,
        labels=between_labels,
        right=True,
    )
    abx_time_between.reset_index(inplace=True)

    # get samples around n-th abx exposure & plot
    n = 3
    i = 0
    score_col = f"score_{n}"
    scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
        abx_scores_flat,
        abx_df,
        n=n,
        min_samples=min_samples,
        max_samples=max_samples,
        group_samples=group_samples,
        score_var=score_col,
        max_resolution=max_resolution,
    )

    for cat in between_labels:
        # filter only for scores of hosts where 2nd abx is in cat time duration
        hosts = abx_time_between[abx_time_between["time_since_2nd_cat"] == cat][
            "host_id"
        ].unique()

        scores_abx_nth_samples_f = scores_abx_nth_samples[
            scores_abx_nth_samples["host_id"].isin(hosts)
        ].copy()
        flag = f"time_since_{n - 1}th_{i}"
        _plot_score_after_nth_abx_exposure(
            scores_abx_nth_samples_f,
            x_axis="diff_age_nth_abx",
            y_axis=score_col,
            n=n,
            path_to_save=evaluation_path,
            flag=flag,
            tag=f"time since {n - 1}nd abx exposure {cat}",
            min_samples=min_samples,
            max_samples=max_samples,
            max_resolution=max_resolution,
            grouped_samples=group_samples,
        )
        i += 1


## Score after 1st and 2nd abx

### split by abx characteristics

#### split by abx duration:

In [ ]:
for n in [1, 2]:
    score_col = f"score_{n}"
    print(score_col)
    scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
        abx_scores_flat,
        abx_df,
        n=n,
        min_samples=min_samples,
        max_samples=max_samples,
        group_samples=group_samples,
        score_var=score_col,
        max_resolution=max_resolution,
    )

    # bin duration into short, mid and long duration
    # scores_abx_nth_samples["abx_any_last_dur_days"].hist(bins=10)

    bins = [-float("inf"), 7, float("inf")]
    dur_labels = ["< 7 days", ">= 7 days"]
    scores_abx_nth_samples["abx_duration_category"] = pd.cut(
        scores_abx_nth_samples["abx_any_last_dur_days"],
        bins=bins,
        labels=dur_labels,
        right=False,
    )

    # print(scores_abx_nth_samples["abx_duration_category"].value_counts(dropna=False))

    i = 0
    for dur in dur_labels:
        print(dur)
        # evaluation_path_bin = f"{evaluation_path}duration_bins{i}/"
        host_w_dur = (
            scores_abx_nth_samples.loc[
                scores_abx_nth_samples["abx_duration_category"] == dur, "host_id"
            ]
            .unique()
            .tolist()
        )
        scores_abx_nth_samples_dur = scores_abx_nth_samples.loc[
            scores_abx_nth_samples["host_id"].isin(host_w_dur)
        ].copy()
        flag_dur = f"dur_{i}"
        print(f"Number of unique infants: {scores_abx_nth_samples_dur['host_id'].nunique()}")
        _plot_score_after_nth_abx_exposure(
            scores_abx_nth_samples_dur,
            x_axis="diff_age_nth_abx",
            y_axis=score_col,
            n=n,
            path_to_save=evaluation_path,
            flag=flag_dur,
            tag=f"duration: {dur}",
            min_samples=min_samples,
            max_samples=max_samples,
            max_resolution=max_resolution,
            grouped_samples=group_samples,
        )

        i += 1

#### split by type of abx (top 1 vs. others) - only n=1

In [ ]:
n = 1
score_col = f"score_{n}"
scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
    abx_scores_flat,
    abx_df,
    n=n,
    min_samples=min_samples,
    max_samples=max_samples,
    group_samples=group_samples,
    score_var=score_col,
    max_resolution=max_resolution,
)


In [ ]:
scores_abx_nth_samples["abx_type"].value_counts(dropna=False)

In [ ]:
# add top 1 vs. others
scores_abx_nth_samples["abx_type_cat"] = scores_abx_nth_samples["abx_type"].apply(
    lambda x: "Penicillin" if x == "Penicillin" else "Others"
)
scores_abx_nth_samples["abx_type_cat"].value_counts(dropna=False)

In [ ]:
# select types to look for
abx_types = scores_abx_nth_samples["abx_type_cat"].unique().tolist()
# abx_types = ["Penicillin", "Cephalosporine", "Cotrimoxazole", "Macrolide"]
abx_types

In [ ]:
abx_col = "abx_type_cat"
# abx_col = "abx_type"
for abx in abx_types:
    print(abx)
    abx_str = abx.lower().replace(" ", "_").replace(",", "_")
    flag_abx = f"abx_type_{abx_str}"
    # evaluation_path_abx = f"{evaluation_path}abx_type_{abx_str}/"

    # age at time of abx exposure should match a
    scores_abx_nth_samples_abx = scores_abx_nth_samples.loc[
        scores_abx_nth_samples[abx_col].str.contains(abx)
    ].copy()

    _plot_score_after_nth_abx_exposure(
        scores_abx_nth_samples_abx,
        x_axis="diff_age_nth_abx",
        y_axis=score_col,
        n=n,
        path_to_save=evaluation_path,
        flag=flag_abx,
        tag=f"Abx type: {abx}",
        min_samples=min_samples,
        max_samples=max_samples,
        max_resolution=max_resolution,
        grouped_samples=group_samples,
    )

    i += 1

#### split by type of abx reason (top 1 vs. others) - only n=1

In [ ]:
n = 1
score_col = f"score_{n}"
scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
    abx_scores_flat,
    abx_df,
    n=n,
    min_samples=min_samples,
    max_samples=max_samples,
    group_samples=group_samples,
    score_var=score_col,
    max_resolution=max_resolution,
)
# top 1 vs. others
print(scores_abx_nth_samples["abx_reason"].value_counts(dropna=False))

scores_abx_nth_samples["abx_reason_cat"] = scores_abx_nth_samples["abx_reason"].apply(
    lambda x: "Otitis media" if x == "Otitis media" else "Others"
)
print(scores_abx_nth_samples["abx_reason_cat"].value_counts(dropna=False))

reasons = scores_abx_nth_samples["abx_reason_cat"].unique().tolist()


In [ ]:
# # top 4 reasons
# reasons = (
#     scores_abx_nth_samples["abx_reason"]
#     .value_counts(dropna=False)
#     .iloc[:4]
#     .index.tolist()
# )
# reasons

In [ ]:
for r in reasons:
    print(r)
    r_str = r.lower().replace(" ", "_").replace(",", "_")
    # evaluation_path_r = f"{evaluation_path}abx_reason_{r_str}/"

    # age at time of abx exposure should match a
    scores_abx_nth_samples_r = scores_abx_nth_samples.loc[
        scores_abx_nth_samples["abx_reason"] == r
    ].copy()
    flag_reason = f"abx_reason_{r_str}"
    _plot_score_after_nth_abx_exposure(
        scores_abx_nth_samples_r,
        x_axis="diff_age_nth_abx",
        y_axis=score_col,
        n=n,
        path_to_save=evaluation_path,
        flag=flag_reason,
        tag=f"Abx reason: {r}",
        min_samples=min_samples,
        max_samples=max_samples,
        max_resolution=max_resolution,
        grouped_samples=group_samples,
    )

    i += 1

### split by host characteristics

#### split by time of life:

In [ ]:
for n in [1, 2]:
    score_col = f"score_{n}"
    print(score_col)
    scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
        abx_scores_flat,
        abx_df,
        n=n,
        min_samples=min_samples,
        max_samples=max_samples,
        group_samples=group_samples,
        score_var=score_col,
        max_resolution=max_resolution,
    )

    # bin age
    # scores_abx_nth_samples["age_nth_abx"].hist(bins=24)

    bins_age = [-float("inf"), 12, float("inf")]
    # <6: pre weaning
    age_labels = ["<= 12 months", "12 - 24 months"]
    scores_abx_nth_samples["age_nth_abx_category"] = pd.cut(
        scores_abx_nth_samples["age_nth_abx"],
        bins=bins_age,
        labels=age_labels,
        right=True,
    )
    print(scores_abx_nth_samples["age_nth_abx_category"].value_counts(dropna=False))

    i = 0
    for a in age_labels:
        print(a)
        # evaluation_path_bin = f"{evaluation_path}age_bins{i}/"
        # age at time of abx exposure should match a
        scores_abx_nth_samples_age = scores_abx_nth_samples.loc[
            scores_abx_nth_samples["age_nth_abx_category"] == a
        ].copy()
        flag_age = f"age_{i}"
        print(f"Number of unique infants: {scores_abx_nth_samples_age['host_id'].nunique()}")
        _plot_score_after_nth_abx_exposure(
            scores_abx_nth_samples_age,
            x_axis="diff_age_nth_abx",
            y_axis=score_col,
            n=n,
            path_to_save=evaluation_path,
            flag=flag_age,
            tag=f"age: {a}",
            min_samples=min_samples,
            max_samples=max_samples,
            max_resolution=max_resolution,
            grouped_samples=group_samples,
        )

        i += 1

#### split by delivery mode:

In [ ]:
n = 1
score_col = f"score_{n}"
scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
    abx_scores_flat,
    abx_df,
    n=n,
    min_samples=min_samples,
    max_samples=max_samples,
    group_samples=group_samples,
    score_var=score_col,
    max_resolution=max_resolution,
)

delivery_labels = scores_abx_nth_samples.delivery_mode.unique().tolist()

i = 0
for a in delivery_labels:
    print(a)
    scores_abx_nth_samples_delivery = scores_abx_nth_samples.loc[
        scores_abx_nth_samples["delivery_mode"] == a
    ].copy()

    flag_delivery = f"delivery_{a}"

    _plot_score_after_nth_abx_exposure(
        scores_abx_nth_samples_delivery,
        x_axis="diff_age_nth_abx",
        y_axis=score_col,
        n=n,
        path_to_save=evaluation_path,
        flag=flag_delivery,
        tag=f"delivery mode: {a}",
        min_samples=min_samples,
        max_samples=max_samples,
        max_resolution=max_resolution,
        grouped_samples=group_samples,
    )

    i += 1

#### split by feeding mode:

In [ ]:
n = 1
score_col = f"score_{n}"
scores_abx_nth_samples = _select_samples_around_nth_abx_exposure(
    abx_scores_flat,
    abx_df,
    n=n,
    min_samples=min_samples,
    max_samples=max_samples,
    group_samples=group_samples,
    score_var=score_col,
    max_resolution=max_resolution,
)

for diet_col in ["diet_milk", "diet_weaning"]:
    diet_labels = scores_abx_nth_samples[diet_col].unique().tolist()

    i = 0
    for a in diet_labels:
        print(a)
        scores_abx_nth_samples_diet = scores_abx_nth_samples.loc[
            scores_abx_nth_samples[diet_col] == a
        ].copy()

        if a is None:
            scores_abx_nth_samples_diet = scores_abx_nth_samples.loc[
                scores_abx_nth_samples[diet_col].isna()
            ].copy()

        flag_diet = f"diet_{a}"

        _plot_score_after_nth_abx_exposure(
            scores_abx_nth_samples_diet,
            x_axis="diff_age_nth_abx",
            y_axis=score_col,
            n=n,
            path_to_save=evaluation_path,
            flag=flag_diet,
            tag=f"diet: {diet_col}={a}",
            min_samples=min_samples,
            max_samples=max_samples,
            max_resolution=max_resolution,
            grouped_samples=group_samples,
        )

        i += 1

## Score over age range

In [ ]:
dic_splits_n_scores = {
    "train_noabx": ["score_1", noabx_train, None],
    "val_noabx": ["score_1", noabx_val, None],
    "abx_1st": ["score_1", abx_scores_flat, abx_age_at_all["age_1st_abx"]],
    "abx_2nd": ["score_2", abx_scores_flat, abx_age_at_all["age_2nd_abx"]],
    "abx_3rd": ["score_3", abx_scores_flat, abx_age_at_all["age_3rd_abx"]],
}

for name, v in dic_splits_n_scores.items():
    score_col = v[0]
    scores = v[1]
    abx_age_values = v[2]
    _plot_score_over_age(scores, score_col, name, evaluation_path, abx_age_values)
    plt.show()

## Score overall - scatter

In [ ]:
# sort both abx dataframes by increasing abx exposure in same way
abx_scores_flat.sort_values(
    [
        "abx_max_count_ever",
        "max_abx_w_microbiome",
        "host_id",
        "day",
    ],
    ascending=[True, True, True, True],
    inplace=True,
)

# sort abx_df accordingly
# sort abx_df in same order and remove samples that don't exist in md_df
abx_events = pd.DataFrame()
abx_events["host_id"] = abx_scores_flat["host_id"].unique()
abx_events = pd.merge(abx_events, abx_df, on="host_id", how="left")

assert abx_events.host_id.unique().tolist() == abx_scores_flat.host_id.unique().tolist()


In [ ]:
dic_splits = {
    "train_noabx": ["score_1", noabx_train, None],
    "val_noabx": ["score_1", noabx_val, None],
    "abx": ["score_1", abx_scores_flat, abx_events],
}

display_scatterplot_w_scores(
    dic_splits, False, path_to_output=evaluation_path, flag="noabx_vs_abx"
)

In [ ]:
dic_splits = {
    "abx_1st": ["score_1", abx_scores_flat, abx_events],
    "abx_2nd": ["score_2", abx_scores_flat, abx_events],
    "abx_3rd": ["score_3", abx_scores_flat, abx_events],
}

display_scatterplot_w_scores(
    dic_splits, False, True, path_to_output=evaluation_path, flag="all_abx"
)

## Individual score trajectories

### abx

In [ ]:
plot_trajectory(
    abx_scores_flat,
    abx_events,
    "P015091",
    ["score_1", "score_2", "score_3"],
    path_to_output=evaluation_path,
    flag="all_scores",
)

In [ ]:
plot_trajectory(
    abx_scores_flat,
    abx_events,
    "P000648",
    ["score_1", "score_2", "score_3"],
    path_to_output=evaluation_path,
    flag="all_scores",
)

In [ ]:
plot_trajectory(
    abx_scores_flat,
    abx_events,
    "E004898",
    ["score_1", "score_2", "score_3"],
    path_to_output=evaluation_path,
    flag="all_scores",
)

In [ ]:
plot_trajectory(
    abx_scores_flat,
    abx_events,
    "E021822",
    ["score_1", "score_2", "score_3"],
    path_to_output=evaluation_path,
    flag="all_scores",
)

### noabx

In [ ]:
plot_trajectory(
    noabx_train,
    None,
    "E035134",
    ["score_1"],
    path_to_output=evaluation_path,
    flag="noabx_score1",
)

In [ ]:
plot_trajectory(
    noabx_train,
    None,
    "E022497",
    ["score_1"],
    path_to_output=evaluation_path,
    flag="noabx_score1",
)